In [2]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
from dlomix.data import FragmentIonIntensityDataset

import os

2024-09-12 17:37:24.792876: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-12 17:37:24.838511: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-12 17:37:24.838536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-12 17:37:24.840397: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-12 17:37:24.848400: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-12 17:37:24.849291: I tensorflow/core/platform/cpu_feature_guard.cc:1


Avaliable feature extractors are (use the key of the following dict and pass it to features_to_extract in the Dataset Class):
{
   "atom_count": "Atom count of PTM.",
   "delta_mass": "Delta mass of PTM.",
   "mod_gain": "Gain of atoms due to PTM.",
   "mod_loss": "Loss of atoms due to PTM.",
   "red_smiles": "Reduced SMILES representation of PTM."
}.
When writing your own feature extractor, you can either
    (1) use the FeatureExtractor class or
    (2) write a function that can be mapped to the Hugging Face dataset.
In both cases, you can access the parsed sequence information from the dataset using the following keys, which all provide python lists:
    - _parsed_sequence: parsed sequence
    - _n_term_mods: N-terminal modifications
    - _c_term_mods: C-terminal modifications



In [3]:
model_name = 'c_i_FiLM_full_d10_static_0.0005_ITy'

model = keras.saving.load_model('/nfs/home/students/d.lochert/projects/astral/dlomix-astral/saved_models/%s.keras' % model_name,
                                custom_objects={'masked_spectral_distance': masked_spectral_distance, 'masked_pearson_correlation_distance': masked_pearson_correlation_distance}, 
                                compile=True, safe_mode=True)
model.summary()

Model: "transformer_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  6656      
                                                                 
 dense_1 (Dense)             multiple                  3328      
                                                                 
 dense_2 (Dense)             multiple                  1315840   
                                                                 
 trans_block (TransBlock)    multiple                  524545    
                                                                 
 trans_block_1 (TransBlock)  multiple                  524545    
                                                                 
 trans_block_2 (TransBlock)  multiple                  524545    
                                                                 
 trans_block_3 (TransBlock)  multiple            

In [4]:
int_data = FragmentIonIntensityDataset.load_from_disk("/nfs/home/students/d.lochert/projects/astral/dlomix-astral/combined_dataset")

In [53]:
#test_batch = [m for m in int_data.tensor_train_data.take(42)][0][0]

test_batch = [m for m in int_data.tensor_train_data.take(42)][1]
test_batch

({'modified_sequence': <tf.Tensor: shape=(1024, 30), dtype=int64, numpy=
  array([[11,  6,  1, ...,  0,  0,  0],
         [ 1, 16, 14, ...,  0,  0,  0],
         [ 8, 13, 18, ...,  0,  0,  0],
         ...,
         [ 4, 10, 16, ...,  0,  0,  0],
         [ 4, 18, 17, ...,  0,  0,  0],
         [ 4, 17,  7, ...,  0,  0,  0]])>,
  'charge_oh': <tf.Tensor: shape=(1024, 6), dtype=float32, numpy=
  array([[0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         ...,
         [0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.]], dtype=float32)>,
  'collision_energy': <tf.Tensor: shape=(1024,), dtype=float32, numpy=
  array([0.2382246, 0.3      , 0.29     , ..., 0.3      , 0.3      ,
         0.27     ], dtype=float32)>,
  'method_nr_oh': <tf.Tensor: shape=(1024, 2), dtype=float32, numpy=
  array([[0., 1.],
         [1., 0.],
         [0., 1.],
         ...,
         [1., 0.],
         [1., 0.],
      

In [65]:
# shuffle metadata
shuffled_batch = test_batch[0].copy()

# shuffled_batch['charge_oh'] = tf.random.shuffle(shuffled_batch['charge_oh'])
# shuffled_batch['collision_energy'] = tf.random.shuffle(shuffled_batch['collision_energy'])
# shuffled_batch['method_nr_oh'] = tf.random.shuffle(shuffled_batch['method_nr_oh'])
shuffled_batch['machine_oh'] = tf.random.shuffle(shuffled_batch['machine_oh'])

shuffled_batch

{'modified_sequence': <tf.Tensor: shape=(1024, 30), dtype=int64, numpy=
 array([[11,  6,  1, ...,  0,  0,  0],
        [ 1, 16, 14, ...,  0,  0,  0],
        [ 8, 13, 18, ...,  0,  0,  0],
        ...,
        [ 4, 10, 16, ...,  0,  0,  0],
        [ 4, 18, 17, ...,  0,  0,  0],
        [ 4, 17,  7, ...,  0,  0,  0]])>,
 'charge_oh': <tf.Tensor: shape=(1024, 6), dtype=float32, numpy=
 array([[0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        ...,
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.]], dtype=float32)>,
 'collision_energy': <tf.Tensor: shape=(1024,), dtype=float32, numpy=
 array([0.2382246, 0.3      , 0.29     , ..., 0.3      , 0.3      ,
        0.27     ], dtype=float32)>,
 'method_nr_oh': <tf.Tensor: shape=(1024, 2), dtype=float32, numpy=
 array([[0., 1.],
        [1., 0.],
        [0., 1.],
        ...,
        [1., 0.],
        [1., 0.],
        [0., 1.]], dtype=float32

In [66]:
batch_prediction = model.predict(test_batch[0])
shuffled_batch_prediction = model.predict(shuffled_batch)

print(tf.reduce_mean( masked_spectral_distance(test_batch[1], batch_prediction) ))
print(tf.reduce_mean( masked_spectral_distance(test_batch[1], shuffled_batch_prediction) ))

32/32 [==============================] - 4s 136ms/step
tf.Tensor(0.16962747, shape=(), dtype=float32)
tf.Tensor(0.39736664, shape=(), dtype=float32)
